In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np

x_data = np.array([
    [0,0],
    [1,0],
    [1,1],
    [0,0],
    [0,0],
    [0,1]
])

y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])


# Deep Neural Network
Classification 예제를 Deep Neural Network를 활용하여 구현한다.

### Neural Network 구성

In [2]:
global_step = tf.Variable(0, trainable=False, name='global_step')

학습에 직접적으로 사용하지 않고 학습 횟수에 따라서 단순히 증가시킬 변수를 만든다.

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

첫번째 Weight의 Dimension은 Feature의 개수 2, 히든 레이어의 노드 개수를 10개로 가정하고 [ 2, 10 ] 으로 정의한다.

In [4]:
W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1,W2))

두번째 레이어에서는 노드를 20개를 사용하여 [10,20]으로 정의한다.

In [5]:
W3 = tf.Variable(tf.random_uniform([20,3], -1.,1.))
model = tf.matmul(L2,W3)

두번째 레이어의 노드 개수인 20과 아웃풋을 3을 Dimension으로 사용한다.

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

Cross-Entropy를 복잡한 수식으로 직접 표현할 필요없이, 텐서플로우에서 제공하는 Cross-Entropy 함수를 활용할 수 있다.

In [7]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

### Neural Network 학습

In [8]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
saver.save(sess, './model/dnn.ckpt',global_step=global_step)

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-2
('Step: 3, ', 'Cost: 1.126')
('Step: 4, ', 'Cost: 1.079')


'./model/dnn.ckpt-4'

In [9]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('Prediction:', sess.run(prediction, feed_dict={X: x_data}))
print('Target:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

('Prediction:', array([0, 2, 0, 0, 0, 2]))
('Target:', array([0, 1, 2, 0, 0, 2]))
Accuracy: 66.67


### 결과 확인
- 0 : 기타
- 1 : 포유류
- 2 : 조류